<a href="https://colab.research.google.com/github/poojasri05-hub/handwritten-digit-recognition/blob/main/mnist_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import confusion_matrix, classification_report

np.random.seed(42)
tf.random.set_seed(42)

print("TensorFlow:", tf.__version__)


TensorFlow: 2.19.0


In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [ ]:
x_train = (x_train / 255.0)[..., None]  # -> (60000, 28, 28, 1)
x_test  = (x_test  / 255.0)[..., None]


In [ ]:
model = models.Sequential([
    layers.Conv2D(32, 3, activation='relu', input_shape=(28,28,1)),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(10, activation='softmax')
])


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy', patience=2, restore_best_weights=True
)
ckpt = tf.keras.callbacks.ModelCheckpoint(
    'mnist_cnn_best.h5', monitor='val_accuracy', save_best_only=True
)


In [ ]:
history = model.fit(
    x_train, y_train,
    epochs=5, batch_size=64,
    validation_split=0.1,
    callbacks=[early_stop, ckpt],
    verbose=1
)


Epoch 1/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8600 - loss: 0.4522

844/844 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - accuracy: 0.8601 - loss: 0.4519 - val_accuracy: 0.9855 - val_loss: 0.0511
Epoch 2/5
830/844 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9781 - loss: 0.0710

844/844 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9781 - loss: 0.0709 - val_accuracy: 0.9867 - val_loss: 0.0467
Epoch 3/5
841/844 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9836 - loss: 0.0528

844/844 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9836 - loss: 0.0528 - val_accuracy: 0.9882 - val_loss: 0.0389
Epoch 4/5
843/844 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9877 - loss: 0.0401

844/844 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9877 - loss: 0.0401 - val_accuracy: 0.9900 - val_loss: 0.0366
Epoch 5/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9898 - loss: 0.0327 - val_accuracy: 0.9900 - val_loss: 0.0383


In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print(f"Test Accuracy: {test_acc:.4f}")


Test Accuracy: 0.9915


In [ ]:
y_pred = np.argmax(model.predict(x_test, verbose=0), axis=1)
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, digits=4))


Confusion Matrix:
 [[ 975    1    1    0    0    0    2    1    0    0]
 [   0 1131    0    0    0    0    3    1    0    0]
 [   0    0 1024    0    2    0    0    5    1    0]
 [   0    0    2 1001    0    2    0    2    3    0]
 [   0    0    1    0  973    0    2    1    1    4]
 [   1    0    0    6    0  883    1    0    0    1]
 [   2    2    0    0    1    3  950    0    0    0]
 [   0    3    1    0    0    1    0 1020    1    2]
 [   1    1    0    1    1    0    2    2  964    2]
 [   1    2    0    0    2    3    2    5    0  994]]

Classification Report:
               precision    recall  f1-score   support

           0     0.9949    0.9949    0.9949       980
           1     0.9921    0.9965    0.9943      1135
           2     0.9951    0.9922    0.9937      1032
           3     0.9931    0.9911    0.9921      1010
           4     0.9939    0.9908    0.9924       982
           5     0.9899    0.9899    0.9899       892
           6     0.9875    0.9916    0.9896   

In [ ]:
model.save("mnist_cnn.h5")
# Later
reloaded = tf.keras.models.load_model("mnist_cnn.h5")


In [ ]:
idx = np.random.choice(len(x_test), 5, replace=False)
samples = x_test[idx]
preds = np.argmax(model.predict(samples, verbose=0), axis=1)
print("True labels:   ", y_test[idx])
print("Predicted:     ", preds)


True labels:    [6 2 3 7 2]
Predicted:      [6 2 3 7 2]
